# Create Mini Crops for the Labeled Dataset

With inspiration using PIL from here https://stackoverflow.com/questions/51379338/image-processing-with-single-to-multiple-images

Plus 2020-03-14-ABCrop.ipynb on sidewalk-cv

In [ ]:
import os
from PIL import Image

In [ ]:
import pandas as pd
#from pano_feats import Pano
import mini_utils
import s3fs
import boto3

# Get list of img_id to crop

# Specify the cropping bounds

# S3 Image Details

In [ ]:
MINI_CROPS_DIR = r's3://gsv-crops2/mini-crops'

# Cropping

In [ ]:
for image_name in files:
    if ".jpg" in image_name:
        img = Image.open(image_name)
        area_top = (125, 70, 444, 253)
        cropped_img_top = img.crop(area_top)
        cropped_img_top.save("cropped_top_"+image_name)

        area_bottom = (125, 306, 444, 490)
        cropped_img_bottom = img.crop(area_bottom)
        cropped_img_bottom.save("cropped_bottom_"+image_name)

# From Amy's ABCrop.ipynb 

In [3]:
meta = pd.read_csv('./data/final_combined_splits.csv', index_col = 0)
meta.head()

,pano_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,labels_count,count_present_ramp,count_missing_ramp,count_obstacle,count_surface_prob,count_no_sidewalk,is_test_pano,includes_rare
img_id,,,,,,,,,,,,,,
352,gmMcgskONK4C-kMOGpQfAw,False,False,False,False,True,11,0,0,0,0,11,True,False
510,2xZABXrvlRIsTW_lb-P-Mw,True,False,False,False,True,4,2,0,0,0,2,False,False
583,7Np-jziLBGYvL0jxef8n6Q,False,False,False,False,True,9,0,0,0,0,9,True,False
878,Rh2vW4whyJGdD6M9v47NcQ,True,False,False,False,True,13,2,0,0,0,11,False,False
944,WeDhagO9OpPej7YAOjAs_g,False,False,False,False,True,17,0,0,0,0,17,True,False


In [4]:
new_df = []
for ix,row in meta.iterrows():
    for heading in [45,135,225,315]:
        new_df.append([row.pano_id, heading, str(ix) + '_' + str(heading)])
new_meta = pd.DataFrame(new_df, columns = ['pano_id','heading','img_id'])

In [5]:
new_meta.set_index('img_id', drop=True, inplace=True)
new_meta.head()

,pano_id,heading
img_id,,
352_45,gmMcgskONK4C-kMOGpQfAw,45
352_135,gmMcgskONK4C-kMOGpQfAw,135
352_225,gmMcgskONK4C-kMOGpQfAw,225
352_315,gmMcgskONK4C-kMOGpQfAw,315
510_45,2xZABXrvlRIsTW_lb-P-Mw,45


In [18]:
new_meta.to_csv('./data/all_panos_meta.csv', index=True)
#new_meta.to_csv('./data/final_combined_splits_panos.csv', index=True)

## Test Crop with Depth

In [6]:
import pandas as pd
from pano_feats import Pano
import updated_utils
import s3fs
import boto3

In [7]:
df = pd.read_csv('./data/all_panos_meta.csv')

In [8]:
# Get list of panos already cropped to remove from list
fs = s3fs.S3FileSystem()
fnames = fs.ls('s3://gsv-crops2/crops/')
fnames = [x for x in fnames if not x.endswith('.jpg')]
fnames = set(fnames)

In [33]:
for i in fnames:
    print(i)
    break

gsv-crops2/crops/5362_315


In [9]:
pano_list = []
for ix, row in df.iterrows():
    if ('gsv-crops2/crops/' + row.img_id) not in fnames:
        pano = Pano(row)
        pano_list.append(pano)

In [10]:
print(len(pano_list), df.shape[0])

4731 77300


In [11]:
updated_utils.make_sliding_window_crops(pano_list)

	 cropping failed for 0_45
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 0_135
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 0_225
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 0_315
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 1_45
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 1_135
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 1_225
An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.
	 cropping failed for 1_315
An error occurred (NoSuchKey) when calling the GetObject operati

In [ ]:
pano_list = []
for ix, row in df.iterrows():
    if ('gsv-crops2/crops/' + row.img_id) not in fnames:
        pano = Pano(row)
        pano_list.append(pano)

In [ ]:
updated_utils.make_sliding_window_crops(pano_list)